In [7]:
import psycopg2
import time
import boto3
import copy
import gc
import pandas as pd

In [26]:
db_connection = psycopg2.connect(
            user='USER_STRING_HERE', password='PW_STRING_HERE', #Credentials removed for Github
            host='mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com', port=5432,
            database="mads"
        )

db_connection.autocommit = True

cursor = db_connection.cursor()

In [27]:
#delete_query = "DROP TABLE greeks5"
#cursor.execute(delete_query)

In [21]:
#db_connection.rollback()

In [41]:
query = '''CREATE TABLE greeks6 (id SERIAL PRIMARY KEY,  
                                exec_date date,
                                increment SMALLINT,
                                opt_value float,
                                ticker VARCHAR(8),
                                greek VARCHAR(16),
                                option_type VARCHAR(8))'''


cursor.execute(query)

db_connection.commit()

In [54]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks5"

cursor.execute(select_query)

cursor.fetchall()

[(0,)]

In [43]:
import pandas as pd

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 
greeks_csv = 's3://{}/{}'.format(bucket, data_key) 

greeks_df = pd.read_csv(greeks_csv, nrows=5) ## Reading only 5 rows to check dataset format.

greeks_df

,date,increment,value,ticker,greek,option_type
0,2014-01-15 00:00:00,360,0.4762,LXU,iv,call
1,2014-01-15 00:00:00,360,0.5087,LXU,iv,put
2,2014-01-15 00:00:00,360,0.4805,LXU,iv,all
3,2014-01-15 00:00:00,360,3702.6400,LXU,gamma,call
4,2014-01-15 00:00:00,360,-420.8000,LXU,gamma,put


In [55]:
greeks_df.to_csv("sample.csv", index=False, header=False)

In [95]:
#pd.DataFrame([["2014-01-15", None, 0.5, "LXU", "iv", "all"], ["2014-01-15", None, 0.5, "LXU", "iv", "all"], ["2014-01-15", 350, None, "LXU", "iv", ""]]).dropna()

In [94]:
#!cat sample.csv

In [93]:
#with open("sample.csv", "rb") as fp:
#    cursor.copy_from(fp, "greeks6", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

In [92]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks5"

cursor.execute(select_query)

cursor.fetchall()

[(17500000,)]

In [59]:
import pandas as pd

select_query = "SELECT * FROM greeks6"

cursor.execute(select_query)

rows = cursor.fetchall()
records = []
for row in rows:
    records.append([row[0], row[1],row[2], row[3], row[4], row[5], row[6]])
    
df = pd.DataFrame(records, columns=["ID", "DATE", "INCREMENT", "VALUE", "TICKER", "GREEK", "OPTION_TYPE"])

print(df.shape)

df

(5, 7)


,ID,DATE,INCREMENT,VALUE,TICKER,GREEK,OPTION_TYPE
0,1,2014-01-15,360,0.4762,LXU,iv,call
1,2,2014-01-15,360,0.5087,LXU,iv,put
2,3,2014-01-15,360,0.4805,LXU,iv,all
3,4,2014-01-15,360,3702.6400,LXU,gamma,call
4,5,2014-01-15,360,-420.8000,LXU,gamma,put


In [37]:
#cursor.copy_from()

In [10]:
%%time

bucket = 'mads-capstone-2023' 
data_key = 'greeks2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

insert_query = "INSERT INTO greeks5 (exec_date, increment, opt_value, ticker, greek, option_type) values ('{}',{},{},'{}','{}','{}');"

long_query = ""

for i, row in enumerate(obj["Body"].iter_lines()):
    if i == 0:
        continue
    #if i<=3000000: ## First Run failed at # Mn records. Condition to skip records till last fail count.
    #    if (i%1_000_000) ==0:
    #        print(f"{i} Records Passed.")
    #    continue
    try:
        row_data = row.decode().split(",")
        dt, incr, opt_value, ticker, greek, opt_type = row_data
        dt = dt.split()[0]
        incr = int(incr) if incr else "NULL"
        opt_value = float(opt_value) if opt_value else "NULL"
    except Exception as e:
        print(e)
        print(f"{i} - {row_data} Failed Parsing.")
        continue
        
    cursor.execute(insert_query.format(dt,incr,opt_value,ticker,greek,opt_type))
    #long_query += copy.copy(query + "\n")
    
    if (i%100_000)==0: ## Batching 100k records and uploading all at once.
        db_connection.commit()
        if (i%1_000_000) == 0:
            print(f"{i} Records Inserted")
        

if long_query:
    cursor.execute(long_query)

1000000 Records Passed.
2000000 Records Passed.
3000000 Records Passed.


KeyboardInterrupt: 

In [ ]:
cursor.copy_from()